In [1]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
from auto.auto_crawl import get_data1, get_data2, get_data3, get_data4, get_data5, get_data6, get_data7_1, get_data7_2, get_data7_3, get_data7_4, get_data7_5, get_data7_6, get_data8_1, get_data8_2, get_data9_1, get_data9_2, get_data_10, scroll_down
import re
import time
import os
import arrow
from selenium.webdriver.chrome.options import Options
import requests
import undetected_chromedriver as uc

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import tqdm.auto as tqdm
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import matplotlib.pyplot as plt
plt.rc('font', family='Microsoft JhengHei')
plt.rcParams['axes.unicode_minus'] = False # 顯示負號
import seaborn as sns
import os
import operator

import math
from collections import defaultdict
from wordcloud import WordCloud, STOPWORDS

In [19]:
FILE_SOURCE  = "auto/new_data"
FILE_DESTINATION = "auto/data"
FILE_PREVIOUS = "auto/previous_data"

In [ ]:
'''
query: siemens, fanuc, syntec, searching name
forum: [Machinists, robotics], a list of str
'''
querys = ""
if querys == "":
    querys = ['syntec']
forums = ['Machinists']

driver = webdriver.Chrome()
driver.maximize_window()

# 獲得 title & 網址
results = []
for query in querys:
    for forum in forums:
        signal = 1
        i = 0
        print(f'{forum}')
        driver.get(f'https://www.reddit.com/r/{forum}/search/?q={query}&type=link&cId=fc7a58b4-f24e-42e5-b633-51c75a99c047&iId=46e77d77-e6e8-4434-94f3-4dcbed0ada05')
        time.sleep(2)
        while signal:
            scroll_down(driver)
            time.sleep(2)
            tmp = driver.find_elements(By.XPATH, '//*[@id="main-content"]/div/reddit-feed/faceplate-tracker')
            j = i
            if j+1 >= len(tmp):
                signal = 0
                break
            for i in range(j+1,len(tmp)):
                title = tmp[i].text.split('\n')[0]
                if i > 1:
                    if results[-1]['title'] == title:
                        signal = 0
                        break
                date = driver.find_elements(By.XPATH, f'//*[@id="main-content"]/div/reddit-feed/faceplate-tracker[{i}]/post-consume-tracker/div/div/div[1]/span/faceplate-timeago/time')[0].get_attribute('datetime')
                website = driver.find_elements(By.XPATH, f'//*[@id="main-content"]/div/reddit-feed/faceplate-tracker[{i}]/post-consume-tracker/div/faceplate-tracker/h2/a')[0].get_attribute('href')
                results.append({'forum':forum,'title':title,'date': date,'website':website})

Machinists


In [23]:
query = 'siemens'
df = pd.read_excel(f'{FILE_SOURCE}/8_競品分析_{query}.xlsx')

In [56]:
file_name = f'8_競品分析_{query}.xlsx'
source_path = f"{FILE_SOURCE}/{file_name}"
destination_path = f"{FILE_DESTINATION}/{file_name}"
df = pd.read_excel(source_path)

if os.path.exists(destination_path):
    DF = pd.read_excel(destination_path)
    for i in range(0,len(df)):
        if df.iloc[i].values in DF.values:
            continue
        DF = pd.concat([DF,df])
else:
    DF = df.copy()
    DF.to_excel(destination_path)

In [58]:
# 9_1: 交易價格
get_data9_1()
file_name = '9_1原料價格.json'

# Load新data
try:
    path_update = f'{FILE_SOURCE}/{file_name}'
    jsonFile = open(path_update,'r')
    f =  jsonFile.read()   # 要先使用 read 讀取檔案
    df_update = json.loads(f)
    jsonFile.close()
except:
    df_update = {}

# 初始化變數
signal_first = 0
path_store = f'{FILE_DESTINATION}/{file_name}'

# 檢查路徑是否存在，若不存在則建立路徑
os.makedirs(FILE_DESTINATION, exist_ok=True)

# 讀取舊的 JSON 文件
try:
    with open(path_store, 'r') as jsonFile:
        df_store = json.load(jsonFile)
    print('讀取舊的 JSON 文件')
except FileNotFoundError:
    # 若文件不存在，則初始化為空字典
    signal_first = 0
    df_store = df_update.copy()
    print('若文件不存在，則初始化為空字典')
except json.JSONDecodeError:
    # 若 JSON 格式錯誤，初始化為空字典
    signal_first = 1
    df_store = {}
    print('若 JSON 格式錯誤，初始化為空字典')

# move_file(FILE_DESTINATION, FILE_PREVIOUS, file_name)
if signal_first == 1:
    df_store = df_update.copy()
else:
    for key1 in df_update.keys():
        for key2 in df_update[key1].keys():
            for key3 in df_update[key1][key2].keys():
                if df_update[key1][key2][key3]['date'][0] in df_store[key1][key2][key3]['date']:
                    break
                for key4 in df_update[key1][key2][key3].keys():
                    for i in range(0,len(df_update[key1][key2][key3][key4])):
                        df_store[key1][key2][key3][key4].append(df_update[key1][key2][key3][key4][i])

json_data = json.dumps(df_store)
with open(path_store, "w") as json_file:
    json_file.write(json_data) 

讀取舊的 JSON 文件


IndexError: list index out of range

In [66]:
df_update

{'Non-ferrous': {'LME-Aluminium': {'LME Aluminium Official Prices': {'date': ['2024/11/22',
     '2024/11/22',
     '2024/11/22',
     '2024/11/22',
     '2024/11/22'],
    'CONTRACT': ['Cash', '3-month', 'Dec-25', 'Dec-26', 'Dec-27'],
    'BID': ['2587.00', '2614.00', '2665.00', '2698.00', '2713.00'],
    'OFFER': ['2589.00', '2615.00', '2670.00', '2703.00', '2718.00']},
   'LME Aluminium Closing Prices': {'date': ['2024/11/22',
     '2024/11/22',
     '2024/11/22',
     '2024/11/22',
     '2024/11/22',
     '2024/11/22',
     '2024/11/22'],
    'CONTRACT': ['3-month',
     'Dec 24',
     'Jan 25',
     'Feb 25',
     'Mar 25',
     'Apr 25',
     'May 25'],
    'PRICE': ['2624.00',
     '2607.35',
     '2616.02',
     '2626.13',
     '2632.26',
     '2634.76',
     '2641.26']},
   'LME Aluminium opening stocks': {'date': ['2024/11/22',
     '2024/11/22',
     '2024/11/22'],
    'STOCKS': ['Opening Stock', 'Live warrants', 'Cancelled warrants'],
    'AMOUNT': ['708550', '396675', '311

In [59]:
df_update[key1][key2][key3]['date']

[]

In [50]:
DF.values

array([['Machinists', 'Siemens or Syntec controller?',
        '2024-10-07T11:14:15.408Z',
        'https://www.reddit.com/r/Machinists/comments/1fy53e0/people_who_use_mastercam_on_siemens_controller/'],
       ['Machinists', 'siemens 828d basic handewheel setup.. Help me.',
        '2024-02-23T08:47:47.042Z',
        'https://www.reddit.com/r/Machinists/comments/1axvuu8/siemens_or_syntec_controller/'],
       ['Machinists', 'Siemens Sinumerik 840D OS',
        '2024-11-15T06:41:47.611Z',
        'https://www.reddit.com/r/Machinists/comments/1grqey0/siemens_828d_basic_handewheel_setup_help_me/'],
       ...,
       ['robotics',
        'What algorithm do servos use internally to move and maintain position?',
        '2023-09-18T07:30:07.570Z',
        'https://www.reddit.com/r/robotics/comments/16loulk/telescopic_linear_slide_forearm_based_on_a/'],
       ['robotics',
        'Releasing Kosti-1 + How to Design Your Own Robot Arm From Scratch',
        '2023-05-21T13:01:58.562Z',
      

In [39]:
driver = webdriver.Chrome()
driver.maximize_window()
url = 'https://www.lme.com/Metals/Ferrous'
# url = 'https://www.lme.com/Metals/Non-ferrous'
driver.get(f'{url}')

scroll_down(driver)

start = 60
end = 90
output = driver.find_elements(By.XPATH, '//*[@href]')
links = []
for i in range(start, end+1):
    link = output[i].get_attribute('href')
    links.append(link)
links = [link for link in links if 'ferrous/' in link or 'Ferrous/' in link]
del links[8]


try:
    updatepath = 'auto/new_data/'
    path_update = f'{updatepath}9_1原料價格.json'
    jsonFile = open(path_update,'r')
    f =  jsonFile.read()   # 要先使用 read 讀取檔案
    DAta = json.loads(f)
    jsonFile.close()
except:
    DAta = {}

DAta = {
    'Non-ferrous':{},
    'Ferrous':{}
}

# for k in range(0,1):
for k in range(0,len(links)):

    driver.get(f'{links[k]}')
    topic = links[k].split('/')[-1]
    for key in DAta.keys():
        if key in links[k]:
            Data = DAta[key]
    Data[topic] = {}
    data = Data[topic] # 傳位子!!
    length = "window.scrollBy(0,500)"
    driver.execute_script(length)
    time.sleep(2)

    number_loc = len(driver.find_elements(By.XPATH, '//*[@id="dataset-tab-0"]/div/div[2]/div[2]/div[*]'))
    date = driver.find_element(By.XPATH, '//*[@id="dataset-tab-0"]/div/div[1]/span[3]').text.split(' ')
    # 取出日、月、年
    day = date[3]
    month = date[4]
    year = date[5]

    # 使用 arrow 解析日期並格式化為 DD/MM/YYYY
    date_str = f"{day} {month} {year}"
    parsed_date = arrow.get(date_str, "DD MMM YYYY")
    formatted_date = parsed_date.format("YYYY/MM/DD")

    if number_loc == 1:
        number_loc+=1
    for loc in range(1,number_loc): # 不用匯率
        output = driver.find_elements(By.XPATH, f'//*[@id="dataset-tab-0"]/div/div[2]/div[2]/div[{loc}]')[0].text
        data_list = output.split('\n')
        for i in range(0,len(data_list)):
            if i == 0:
                name = data_list[i]
                data[name] = {}
                data[name]['date'] = [] 
            elif i == 1:
                if loc != 4:
                    columns = data_list[i].split(' ')
                else:
                    columns = ['CONTRACT', 'PRICE']
                    if k!=0 and k!=2:
                        values = data_list[i].rsplit(' ', 1)

                for column in columns:
                    data[name][column] = []
                
                if k!=0 and k!=2:
                    if loc == 4:
                        for j in range(0,len(columns)):
                            data[name][columns[j]].append(values[j]) 
            else:
                if loc == 2 or loc == 3 or loc == 4 or k>=8 :
                    values = data_list[i].rsplit(' ', 1)
                else:
                    values = data_list[i].split(' ')
                    
                for j in range(0,len(columns)):
                    data[name][columns[j]].append(values[j])
                data[name]['date'].append(formatted_date)

json_data = json.dumps(DAta)
with open(path_update, "w") as json_file:
    json_file.write(json_data)   

In [5]:
updatepath = 'auto/data/'
datatype = '9_1'
query = '原料價格'
path_update = f'{updatepath}{datatype}{query}.json'
jsonFile = open(path_update,'r')
f =  jsonFile.read()   # 要先使用 read 讀取檔案
df9_1 = json.loads(f)
jsonFile.close()

In [6]:
df9_1['Non-ferrous']['LME-Aluminium']['LME Aluminium Official Prices']['date']

['2024/11/20', '2024/11/20', '2024/11/20', '2024/11/20', '2024/11/20']

In [7]:
df9_1['Non-ferrous']['LME-Aluminium']['LME Aluminium Official Prices']['CONTRACT']

['Cash', '3-month', 'Dec-25', 'Dec-26', 'Dec-27']

In [34]:
df9_1['Non-ferrous']['LME-Aluminium']['LME Aluminium Official Prices']['BID']

['2503.00',
 '2524.00',
 '2572.00',
 '2617.00',
 '2652.00',
 '2582.50',
 '2607.00',
 '2645.00',
 '2655.00',
 '2660.00',
 '2654.00',
 '2672.00',
 '2702.00',
 '2702.00',
 '2702.00']

In [ ]:

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
}
lastet_month = '11309'
function_type = 0
if function_type == 0:
    '''lastest_month: 11308'''
    payload = {
'__EVENTTARGET': '',
'__EVENTARGUMENT':'',
'__LASTFOCUS': '',
'__VIEWSTATE': '...',
'__VIEWSTATEGENERATOR': 'D99B2448',
'__SCROLLPOSITIONX': '0',
'__SCROLLPOSITIONY': '299',
'__EVENTVALIDATION':' J7NXjkqClbAiAFZPJq85cmZPs6fvJ2Opm1mB1bdbE4fqw17O3kw/NLnZ+T2aJ0hMlS8u5oJyf3XTkAtxrWrFIUX0w+jUvSggqejvBW/5XMvLnHmtoIw7SWwmnI9W5TCFXLC9ID7y2F6UuAuwdqZTPu1JICn1H4q23qYetZ98sGPRllWV/FY1ek+0YeAftpEKD+S5exwqP459Jl6BeUuJ6v84+H7USYjicsG3GHY+vx+bpwak4piV8cOqDzQPJH9xOQt7Dwq1VCrJFxoJF6/YCa6jjpdjQrNW9bbRBvqm6suvYbU4ZzLmYoLXYcotZnL65ql/wpgNdJ1f83+h6cPA5PRBV/gG0N5ma2CrJmHmCyzPvVzCrjgb5XmhxhOmFXPY0F1oLpS3BRYx/LaYWLZso/YRjpFdcJcZT2KiW90+/JARlerD8G53wGGBJwYKWZom22szQyEK1DloZm2vyRQVwm96nJb01URC3SaMazxQy72qou2JnFoR222/JpXmWElg5AmNxLW8YTO9FID0taCBHDWYAwURv0YNMPRsdQ08Abnx0sIZsaP2RtxCZcBMHXIRCxBb3O41+I06qmfQYsxGQZHq3TPcO1cLIYnC5tNrhLdcnGMNperDAXraw4Z9uxe+wckbIh4Yvz0aSTpf9a7Mf3IQpvz9Ita9kMcYPsEABWqqzjYWOR+u0y2owb2/tnLiUVMUsE26OYvwHVYDQrRKGhR1rZ6spV4DCIiZ9RRFWXYLEc1TsvIoXVCtQi7XBO4iFGKmlNqAgyhntO4JH6o+IGcxqMZc4s5yOPZ5BHzKt20gZcaiwTNpkLef+vbm2n9wMUL54NVnNivWAf6oX9lPVi9u9ovQoqIEU+e+qdLlewU9gkFFYjdEDH6iRIzBxMvXRAL+S4wpViy7h2ICg1T2tEDUHW45uGWwZ52aN8gr4tzHPfl7i29SymRUzanyEC9MkpCexZ0xAHVfIdEAmqPvcd35s+JKWWkBsSs3xF4lZzvbMKSdnhO6OOeKEMf216u+GD5awp2rqzM9fPAZFwp0OAJcro3ayt4t3azrzZQpWEvyCTcCdFg8uCb8pTj7/6CBozMkPM9BGrnlFgncIin2JZX8KUyBoVF7s1hhGLZdTeQ16gH4kgSsHBA70NY0B9BQRSMU5NS/BJe4/PrrPzURS2yTRKmZOdZaLscSF2lCu+WKYLsi2qIzmG9afjknOmpDaSp8SBaDdGKhyBiZoudoi71QW+dlHvF0zv5i478zqKvNhXunxTq1ZitMvEJcV8897WTpC9lLvPHraOsHpXGRpg60ZFeRuncRiXYCs70e+d1QMSD+FVmCZ2D+BmU+l4HUIHooY7KrhG1OVADhsWtfHgj15g5CXPh3psPq8oaXRAzX0A0kmpRmYYq29drygE226D56wqk5Lkd/C7cMlette8po0H2gw+3/NxiGwOLVi+BgIKBtW6QMxHGB71HTWeh2gKWgCVG6dQgvNCiQLSaFUsgMu5qAtmqzQtqKsevTP8ZDHvSQRTmgMSSesPachA5KmRkQIrBcGRTB4mOiH2hERf69QWfM4pwoXIdN/Lp7MkFfZ/+F2NLh4XwKnvtkP2YnpVhRV/ZXj8rZLZQBaAtXglOKH5ToGXuTnd8JUND+FNXcFkfAQKeJzEoe5U7sS3DiuDIV6pXWq19eklD2ZkbG9Y6eSLAXW3cNgCCyvRfCIfcDo2hNDky1wDWq/sklLXpGD0hYeatsy35iY9hiChyhNMsoLVAwDF0aBZEXkk7epd088kiebgWdMCpNCCmI6cpbs/HNwVnJ3swJMpQuRwcWunGRVGT2Et86Nf9Z+1R0wjxKUpOCW7LPIJnROgeHVVj+0WNz9j1Yy+Es7afpEn2k1GJM9HZRi3srI34I15RAPKhm4LYQYxVqoWQE/AkFQjwwdorzMmBnDiMDLQ3btG347r1Z6jI+e3JbJiW787lGysLlIB9I5KVcUYa+SdpBgZu8kr7r9daEmdPErkbTWfGf/rVpAlmcXi0aCFS+adzCMZkFI9vevNvEX8K5631AqnBg5A4n9UApoFe+L3kNRuZP67D0VRdam3BHxO4pBDkN4fSCJ/eFTMF4rZYGPAlj16Sur1sryaF+sQHT+i7BTDb+6gqNwhguIp2DlVVqOU9WfhRtjBA14WAVIR+iwbwJ2jd6owdszlilmxf8TgWgrtxcHTQ5VmCT54JWxPzP1KnqVhuYWxAdttSuwIXWXNCpTW/ylvhWY0gXh5B1DlSZosiijbjSjl/q2qGbORTHUIqefhmCrM1fy+OOARE+rTKGrTYYwt23ioprMPhYZv+tVH4/yVoGEYE83jg4vVhKNfIMsttPrcKNsIMko6/3kyuGnKQ0G1+cct0fxVMOBgzJCsIj35WEwtdww2p+dKckNwHWJ30HD5NOOl6pCfCoYA1XBOdTmWCooS8zEf0TrfEdMV/6XJnAZmkdCPq9u4Eh4Rx2cg977lfEPjqtIqw3xln8emomlsBLhrboDjp7q2M6dgQPeAd6M0w4xM144kL6VwQMSGjjLlKuIxgQgJsFdozsDLDfd5jOOWHECVQ8emticxvR2koQnyTMvg7C1mCnNec6evEiCBaMm+jCP8gYoIUJSLWqCdoXhFBBJZYUVwQcfSd6EHWCM2TsV0rUSHrDAoR4BfwXyTn48aDK53VvbYE+cdidLsAMU+T2nFZV3dwtYwrcU5y1VLzIjJX+1sC42uGmVKusO8MsujplbOch0m/bO0/ly5cdfScN2YD7V5c36wioeJZx/UClQrOXP8+JKs3549tNRkCMvkuIqCX093KG1ALMgEUcDXzPfYgQvzZ8yJae9w5nWhOGYOGASHNhapXDTO17DdqgldKr7yG2nz5gN/t6J5w9nWgGnoL1g9Xao0gUuGaA84qbyD2hPX3hT8YrrHhGA0eTVxghuDE91K6BCkXhxM/gl86pgdPCZZ0XbMJ8z5nIWId0HUsi/gQBu5w+oGCZk/Jm2BkoMq/fFfxivhO+B2K0TuJwUbJ73cDdCITzhq6qYsCJnpV4bJGvJGEYgYy1B0Lf4cOnAUsiMFAiVG++uxyHCSvE6bn4tsFDWwqXqYEb0Xh15V0KL8eNWdf4weg/U6QOZUO2X1jHcyzZJG9WXR1Zcr8PD9UD8lKwWq/qlKUxsZsddC21bOD39wzuIBU+Y6yAqGusVT5fshM3mZUq5C4t9iireXq78dlMZlhleoDkgHhpXo30jZLbwBujS6mSMRhZBycmd39k4KHU+tb0aCbPdXJihkN83ziB0scJG0ZR22PNnzWiJ7gVf6LuRLENzZ8w5+SMu49F4lc7w16SybUM8o8PY9VLazvdE+ckiBwXXjzAOsaaP5NrHtco9yJiJQzfwvWdZz+N4JCPWKZ6U5W9BxbS/cabQw6sxyafsQGd7Twj8vsuEcyq5yKzG2dO3Ztak2MWeUzH2/MGE8WqxNGNNB8RMex9ZZGdZOMJjImcV518T7REIvfZDYhbXkagQJiZ/E+qmEWk4HCYBoF8Bn6NSSeNwMY/kthe/uG/bmqIkxaR+CZJ+eGBWo/awr1u308E4+2OXY11BVWIHf4lniW7umWNT/I74u1VK6BHsQE4/JYFstcQz8S6+Koy76yWFggiULvporXu6h8mB7KHU1oEz6NzO4pIc521/lkJUG021f8pYJBI5kCLmwMoEpcGJAPKj6p4JIm1X8LMReOSx1/YfFY0EWXKgqlSZsBRzM8J9VUg4NrmDgke4ngqq5asFtdvFEfF4CKa+IOb7FoY3SZytHBvYYnxhCaf4iFdi0BUKgW5E2I8sCCnh7wcAEViSSX9HDhf17tCIVqUKGRzZw+wIhvHZVV8zWQbw2A3+t0qrIwsi0N9sOhVQ0RbzApxiZoUDt2uXzYKwmv/HIl748GheABM/fxy5jECWtUncQeR/9enwrpntMpSWv+h88/CWmXHNJ2Ls48biNWY78ZOzuyEnidN4vZeMOdZ5Mk0wNTcIZUDxzk04+1tcZvwskPWPUQPS8WLZwojdnP+vlgoOmDdW3SDCAlBOoBMl4q/hJPnYMfQb37lLpU6jYRc80+aKqO08jM0Roi6ALHMRjmR9BZakdLR9Ukxse1nz0Lo/8USZwnej50tE+uemq0BtyqgpHhd2vOGNsphczwWpcUqJwk4cxoOM1UL/aavlkGj+GiX8pQRjtKVVLBRsdHwFvYs5d8rj3Lsxswma9iEEVDLeAn+QUMMb/W7VH58d+Z7Sd5B6jlQkbKvOMIgIxIwI+Jz/IuSVd1w+1oHaiJNM9a/8KWNuqPHMDvOdvjHUCmXJtM+spAnu93P3jpXlX/Pt20PM2uEsaN9knrp9gDM+Ga7eDjWl6fGqSMriCnzVC7umZ+rvLag1WvHdcdxjgAbmbKglgGJnLb3FxoZ4lg59Bq6EGkjH1tncyH89LcTNGGHpSteReNoJ/RTSChCzVZv3xGNI9eO9ygNqrYl0UO6yYYCkX4/z+54UCsr0qxa6iLXHAw4u1qUSV+NNUcHI8YT6Ln3A2SnQRMQPW9ZcycuCDXZYhvrTLeoXidv14wJ1MP6lXkK6TGVI9XggNfRUP+rEOkkbrxtsRrZ1CzX4bpnfpLx9JEPWsuu/8MB5Ry/X/u6Ds3+QOcOHEb+SpOg5wMoFIiyy2oChn35yHTXmJHAIdLAFEU/cYV1QpTktpoQMxI64FKuZz5fFv5f5f3jqRjg7QXHp1dnC3dNe83pE2Nwqt27NQ1UWazq9R1VMOOi4h50B6NPtOHvA4J1EWUC6mEs98BqfnJSZdJA6cW8yoVL3VRPKX2g5F5SLIIHWKv2OTaqt0CwcKdGrDEKkUV+0nYHx8lwSldL9FEVggDCMz9Ftq3z2bf2H6xM6kjB7GEtyagWTX/U6sFafUbztYdPq3pcJNHrvyhwDF4Fb+Uvik0672LRL+p7R8har5hzZbVZ9voeugRkCUASRSkXRwW7JX3ob0g66rGWybRL3OWooX8O0Xy49ZP4Xee4uErHo0JEjw6TtcBy52860Pe0cSpJJhk9yNa2JJKEAVv6dIrlVvWJSVi1skG/1Jysws6n8Km6izyNtfLIlRpktjur9VftmtEfdOCWd83MIx67djC/fMe2HBwgawm+FaIsrg7+cv1nt0yDK5avOPAorq3ILW6ocZut2a7PgTlRKgHCsmJ0pDxtTWyUHs8W/bNU50uh2ojY6oiEu6aYRzVHNMQOez2MboDw4zpf7/lz9YAYhPvGrvICdAMSxLVf0X5mgF64O3N7kVyisxqYbgGPPQeUraqdX7YtgEsRB+AlL9RZrfHMORfFNA16coPhdxjFiAy6lwlXqOMikMeXa/6lEJYH+1OcV3QS5pMuKx6F3/TgS2laHSwKuROMl7EK+ExhG1gxLT+SVKUp9yFBi+N2M8bX9GaBX5s9fD3ROhOuYcxZjD67knO3y+EXOK9OpAVRe0DMa5fl9dE4yEpIYoXYgg89Rd5x0lpCsBtyH9TuUwcKOhn8mDieXuL98psewopveHOEmgNx+dn6iVAPLCGr7iX7c9WdRL6MnDbDSe4wLHmQijkTHpcyuDQMaucssSgnkWF4og1IYSUsRYO4yWVcDwy1ZASu4xRn/Tgt2BexFv+QjhnHV6qVci+VJLFSXlK9fpLK7yqyCnK5XtAL/Rr/huaMHhBShaI0aDIzJOTm1HrvYe3zV2v6ZhA/U0vBllV4I9BDqfvW8AfQYmkNnGJpz97DjuxCu85UsNdczWLMILzcuYcihBV3MoCTrSIqlguz+MFfc4+g9jsdfrederpDDScqIgEWJHsNxk7JgV39FBWcAQL64iBM0XCXj+Ycz/tENuziYk6IntSaClXsnltcj4p4U+O958w+0WjsynvpdrGzw0ojNUNgXsCz/WDfB30/7Q+is/dL88mf46i7bsi0oPgg1j2wivZpYHmmU4yaLCwMTblUyngG5MWHqSesjMbrMbPTCeD4qBsiiV2n2ArU4kEityDR7ySJc7WYrCUKunOP1A8qgAZkzgyguPHbM+ZUgTpj5iYKa3ojE9aj3Coqft02KxUzYzqNy5T0os7O3hsuTS9o10m0FUCrM/MlR2aQ/UgLVZInlhnj85h2lS9W0lAMYw6yy+qokuRAbwJn/1wg3ctFNV+u+GWangQUwWQU+poGdE9a7SiXLIsdh8CrzMbpBfcSCk9SeRqWhScV8Wcg26W89y1DnPvNcWTMQOYneE+Ry+EO53+zRA5h2hWV33s5O87mOI8oBlaFKCFVh1QZaYE6Ryp+5oBYqx82Q5L7KkSnwTNYrm3D1IUTUCTjfsKL/MoB1w1aci3Hljuxz64L3MLbp63emhOqSh8QMEAXmhNEizH6AzYz9y23hq3pGGB7HvGEYckHWn0oDshJH3enoMgmpaNULi6D4L6Vk21rRAz0W4aVKW5ZLCMY/4ypra1imTzThwNKpSUM7C56St9Bqm+N1q8rpSyeez/0eGtcW5nmNPI+kdCA2ejXKhNirftYbyF6/Qi1YkaZZBXvAlYLfDIFEI5/wsm7YQYnXmgcDysd4SXjk4CwiDAr6hSNr5KNIGstJaML2omV52iQQVyqmAIlwBeF+lxIkYZywga/E57I3Fsrbb5HGzcBzWX45ZUyNZkV2GZU23HGQvSn7tEFxGx9TMZysGMooplTlamrnCUS75DujlgvZjl7yAH1RWrhvNYTeurgOMgMveOI+g4zs8NZQvSI5CKde0Z/3KpwfHPZSCLFBk+nkY5c9+mAq9DLl9NHNqMFV3s2OD1brcteLbzy/BWeRkpgBe4nNOYattck4i4rh6gRqelnBYP8kLdwNa1jTX+yhuavwqmzDRuUwz8tbahHTTczcGi/DbLHH5QMeCJEHdl5hrb/35Trmkp5uVUhzgpwU13xiUAnDGAk2hz7OPhYsQNeuOa8TUJjFm1IkBFX4zkdblKvVfqGpcDc+hd6SDgIZW89ilFSPATLOVeo1xFW6nzoUA7m1O28YxcFgF2G/exiz/cnHoMI8VG7fI2GYsFX7lFNOiFfAe6P566IuNKNrINlngr8VtykFsrCZYAXjPXDHL/QO6lZ7cdaADTGifnsSPCO4WdD82YMXHU5gHS5NunHtC1WlxOe1pXDL2wxXXQj710sF5Hl1G/IY4g6aviOldUMyDeJFTvqRY21Us7jOCgp9aED4HszB/SnBDb3XjNpGNsU89D7kQUBy08nohreqAV+FhxDUugs39tTCUxFljmpZGy3EMaxbrDUAwDI2CQPeAisw7tzwZJgVphRhw6D/Hf9zr4IUj2EHUA34QDYaDYTcmqB++QvG6IlV2V8UM7VKJaVv1/Drf7vUlQ2VROtjVuJ/u8F50S3utolWA/vJ0hEbhrRDu47RIkPlKVE3ojnsmvln29Dk9aesgrs8Wpf7tEvJ2md/D1MgVM9LTk7MGynT5C8huGudCYmutkvM167+8kpB5Q7xICK5bwyXUiH2OmakJ7UCotl9Y5WCodkEIOz5jRox2WkjqbVCgCVfe0gQfsftBZYnHt56/7m+aEHHea+VsafSnydnixrRK7Y1tdJg/maDNgGAoXBaXmih8xwFg/bGUKk+S0W5dTa/Nc8Ll7xNyS2Ra/X50EDhEmzeBXV9F57gIUZUvKdE/VoQo4a8hZKYqtIJgDdpYmYwp/pBVBpoMp6EjugCooIZvz81aHfjvpzQVo5Dk77Pqw5khMBEvGG9jT2fubHz7FAhsr4OKmBRe1rS1GJD8fqmUF2DC4NrESiWMZC8h6tOfYAovUJl7FRd8A1qdx1TLJb+yPzTFNMBSVsQa/BLTDHmDBFEv2ysSvqF81aqRAju5ETclkzVhpdqnMPUBc5FEBegHpVryQLBftWn5aaWcvhJ1ao6dQK3TKrzBkyzFRqp/dRaqkukEFL9XM9gH2/nxZ4TGYdgd5nbvzZn0bSTp0OscyGmI/25iEY5wRLvgN/0ah8dcLS2trNuacGhnP8gs9trWXbNuQJFSA9Zo1EPFrekPf30vSiC0LfDVdhT14b/wGFSU4k1pw9eLRnxymC/q5PmoYEOxHJQeVTL55pvkzVkv4JJgsKf+0MaS/ePGbcx9fw1NLbWMXN+ZfvGuTS9dH5VkNpc8/IRn629mmMTC7sBjvW/Ec67DggBp2kzrZzLSIxHvcC6y2IzMfsiyBVIOGy+JxBdz/qdrDi5xa061Z4wzth8n3P1hAdiIgVkxqQwOezud/+pCWql29r/1PobSkU1gRQ6YM=',
'ctl00$hldContent$ddlPeriod': 'YM',
'ctl00$hldContent$ddlDateBeg': '10001',
'ctl00$hldContent$ddlDateEnd': '11309',
'ctl00$hldContent$hidColCount': '2',
'ctl00$hldContent$hidRowCount': '3',
'ctl00$hldContent$ddlDateKind': '民國',
'ctl00$hldContent$ddlYear': '112',
'ctl00$hldContent$ddlQuarterMonth': '10',
'ctl00$hldContent$chkYear': 'on',
'ctl00$hldContent$ddlYearLines': '0',
'ctl00$hldContent$chkMonth': 'on',
'ctl00$hldContent$ddlMonthLines': '166',
'ctl00$hldContent$btnQuery': '查詢',
'ctl00$hldContent$ddlReportType': 'XLS'
    }
    res = requests.post('https://dmz26.moea.gov.tw/GA/common/Common.aspx?code=N&no=18', data = payload, headers = headers, verify = False)
else:
    res = requests.post('https://dmz26.moea.gov.tw/GA/common/Common.aspx?code=N&no=18', headers = headers, verify = False)

soup = BeautifulSoup(res.content, 'lxml')

# Find the relevant table
table = soup.find('table')  # Modify this if the table has specific classes/IDs

# Extract rows
rows = []
for tr in table.find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    if row:
        row = [item.replace(' ','').replace('...','') for item in row]
        rows.append(row)

rows[1] = [item.replace('\xa0', '').replace('\u3000', '').strip() for item in rows[1]]
rows[1].insert(1,' ')

# Create a DataFrame
df = pd.DataFrame(rows)
df.columns = rows[1]
df = df[3:].copy()
df = df[:-3].copy()
df = df.reset_index(drop = True)

time_list = []
for i in range(0,len(df)):
    if '年' in df.iloc[i]['年度別']:
        tmp = df.iloc[i]['年度別']
    time_list.append(tmp + df.iloc[i][' '])
df['日期'] = time_list
if function_type == 1:
    df = df.drop(df[df[' '] == ''].index[0])
df['日期'] = df['日期'].apply(lambda x: str(int(x.split('年')[0])+1911) +'0'+str(x.split('年')[1].split('月')[0]) if int(x.split('年')[1].split('月')[0])<10 else str(int(x.split('年')[0])+1911) + str(x.split('年')[1].split('月')[0]))

del df['年度別']
del df[' ']
df.index = df['日期']
del df['日期']

df = df.applymap(lambda x: np.nan if x == '' else x)
df = df.astype(float)
df = df[df.columns[:-1]].copy()
print('完成爬蟲!')

ValueError: invalid literal for int() with base 10: ''

In [21]:
FILE_SOURCE  = "auto/new_data"
FILE_DESTINATION = "auto/data"
FILE_PREVIOUS = "auto/previous_data"
DF = pd.read_excel(f'{FILE_DESTINATION}/3_主要國家工業生產增加率.xlsx')

df = get_data3(lastet_month, function_type = 1)
df.to_excel(f'{FILE_SOURCE}/3_主要國家工業生產增加率.xlsx')
df = pd.read_excel(f'{FILE_SOURCE}/3_主要國家工業生產增加率.xlsx')

for i in range(0,len(df)):
    if df.iloc[i]['日期'] not in DF['日期'].values:
        DF = pd.concat([DF,pd.DataFrame(df.iloc[i]).T], axis = 0)

完成爬蟲!


In [ ]:
DF = DF.sort_values('日期')

In [24]:
DF

,日期,中華民國,美國,日本,德國,法國,英國,韓國,新加坡,香港,中國大陸
0,201101.0,15.05,4.6,4.4,15.1,7.9,-4.4,11.9,25.6,NaN,NaN
1,201102.0,10.66,3.7,4.0,15.8,7.2,-7.9,6.9,11.0,NaN,14.9
2,201103.0,10.64,4.2,-13.4,9.8,4.1,-8.0,8.8,20.1,3.5,14.8
3,201104.0,4.73,3.4,-13.4,5.0,0.1,-9.5,5.5,-7.0,NaN,13.4
4,201105.0,5.62,2.2,-5.1,17.8,12.8,-3.5,7.1,-14.9,NaN,13.3
...,...,...,...,...,...,...,...,...,...,...,...
160,202405.0,15.73,0.1,1.1,-9.6,-3.4,-0.7,4.2,2.3,NaN,5.6
161,202406.0,12.82,0.9,-7.9,-8.3,-6.1,-6.6,3.8,-4.0,0.6,5.3
162,202407.0,13.01,-0.3,2.9,1.5,6.3,2.7,5.2,2.6,NaN,5.1
163,202408.0,12.54,-0.1,-4.9,-6.4,-1.6,-2.9,3.8,22.0,NaN,4.5


In [8]:
get_data9_2()
file_path = os.listdir('auto/new_data')
file_paths = [path for path in file_path if 'ferrous'in path or 'Ferrous' in path]
filenames = file_paths.copy()
first_time = 0

try:
    updatepath = 'auto/data/'
    path_update = f'{updatepath}9_2原料交易量.json'
    jsonFile = open(path_update,'r')
    f =  jsonFile.read()   # 要先使用 read 讀取檔案
    DF = json.loads(f)
    jsonFile.close()
except:
    first_time = 1
    DF = {}
    DF['Ferrous'] = {}
    DF['Non-ferrous'] = {}

for file_path in file_paths:
    df = pd.read_excel(f'auto/new_data/{file_path}', index_col=0)
    cleaned_filename = file_path.replace('9_2_', '').replace('.xlsx', '').split('_')
    if first_time == 1:
        DF[cleaned_filename[0]][cleaned_filename[1]] = {}
    data = DF[cleaned_filename[0]][cleaned_filename[1]]
    for i in range(0,len(df)):
        if i == 0:
            columns = df.columns
            for column in columns:
                if column not in data.keys():
                    data[column] = []
        if df.iloc[i]['date'] in data['date'] and df.iloc[i]['CONTRACT'] == data['CONTRACT'][-1]:
            print(df.iloc[i]['date'])
            continue
        for column in columns:
            data[column].append(str(df.iloc[i][column]))

2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18
2024/11/18


In [5]:
df.iloc[i]['date']

'2024/11/18'

In [6]:
data['date']

['2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/11',
 '2024/11/13',
 '2024/11/18',
 '2024/11/18']

In [3]:
# 參考字典
tmp_ref = {'世界':['世界'],
'人民幣':['中國', 'RMB', '人幣', '神州', 'Shibor', '央行', '人民幣'],
'美元':['美國', '紐約', 'Fed', 'USD', '美聯儲', '聯準會','美元'],
'台幣':['台灣', '台海', '訪台', 'TWD', '台幣'],
'日圓':['日本', 'JPY', '日圓'],
'澳幣':['澳洲','澳幣'],
'英鎊':['英國','英鎊'],
'歐元':['德國', '法國','歐元']}

value_tmp = list(tmp_ref.values())
key_tmp = list(tmp_ref.keys())

df_ref = pd.DataFrame()
for i in range(0,len(value_tmp)):
    for tmp in value_tmp[i]:
        tmp = pd.DataFrame([key_tmp[i], tmp]).T
        df_ref = pd.concat([df_ref, tmp])
df_ref.columns = ['市場別', '名稱']

df_ref_dict = df_ref['名稱'].values

df_ref_dict = dict(zip(df_ref_dict, [1]*len(df_ref_dict)))

#產業 and 項目
sectors = list(set(df_ref['市場別']))

#公司
companies = df_ref['名稱'].values

dirPath = r"Forex_dashboard/原始鉅亨"
input_files = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]

# 切詞
ws = WS("Forex_dashboard/data")
pos = POS("Forex_dashboard/data")
ner = NER("Forex_dashboard/data")


# 建立一個詞典，詞典中的每個詞都有一個對應的權重
word_to_weight = df_ref_dict

# 使用 construct_dictionary 函數來將上述的詞與權重建立成詞典
dictionary = construct_dictionary(word_to_weight)

# 一定要切的標點符號
segment_delimiter_set =  {",", "。", ":", "?", "!", ";","〈","〉","，","(",")", " ", "、"}

target_filelist = os.listdir("Forex_dashboard/標完產業與項目的鉅亨_標題與關鍵字")

for filename in tqdm.tqdm(input_files):
    if filename in target_filelist:
        continue
    df = pd.read_csv('Forex_dashboard/原始鉅亨/'+ filename, index_col = '日期')
    df['標題'] = df['標題'].astype(str)
    sentence_list = df['標題'].values

    word_sentence_list = ws(
        sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        segment_delimiter_set = segment_delimiter_set, # This is the defualt set of delimiters
        # recommend_dictionary = dictionary, # words in this dictionary are encouraged
        coerce_dictionary = dictionary, # words in this dictionary are forced
    )

    pos_sentence_list = pos(word_sentence_list) # 詞性

    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

    ## 找標題中的名詞
    related_company_ls = []
    df['標題相關詞彙'] = word_sentence_list
    df['關鍵字'] = [tmp.split(',') if type(tmp)!= float else [''] for tmp in df['關鍵字']]
    df['標題與關鍵字相關詞彙'] = df.apply(lambda x: x['標題相關詞彙']+x['關鍵字'], axis = 1)

    ## 找標題中的名詞的公司 & 其產業
    company_list = []
    sector_list = []
    for i in range(0,len(df)):
        tmp = df['標題與關鍵字相關詞彙'][i]
        tmp_list = []
        tmp_sector_list = []
        for j in range(len(tmp)):
            if tmp[j] in df_ref_dict.keys():
                tmp_list.append(tmp[j])
                tmp_sector_list.append(df_ref[df_ref['名稱'] == tmp[j]]['市場別'].values)
        if len(tmp_list) == 0:
            tmp_list = ['未標示']
            tmp_sector_list = [['未標示']]
        company_list.append(tmp_list)
        sector_list.append(tmp_sector_list)
    df['標題相關公司'] = company_list
    df['標題相關產業'] = sector_list

    # 擴增, 有些標題有許多家公司
    DF = pd.DataFrame()
    for i in range(0,len(df)):
        company_list = df['標題相關公司'].iloc[i]
        sector_list = df['標題相關產業'].iloc[i]
        tmp = df.iloc[i].copy()
        if len(company_list) > 0:
            for j in range(0,len(company_list)):
                tmp['標題相關公司'] = company_list[j]
                tmp['標題相關產業'] = sector_list[j][0]
                DF = pd.concat([DF,tmp], axis = 1)
        else:
            DF = pd.concat([DF,tmp], axis = 1)
    DF = DF.T

    ## 將產業轉dummy
    for sector in sectors:
        DF[sector] = DF['標題相關產業'].apply(lambda x: 1 if sector in x else 0)

    DF.to_csv('./Forex_dashboard/標完產業與項目的鉅亨_標題與關鍵字/'+filename, encoding = 'utf-8-sig', index = True)
# 釋放 WS、POS、NER 使用的記憶體
del ws
del pos
del ner


 99%|█████████▉| 259/261 [00:00<00:00, 7860.13it/s]


ValueError: Index 日期 invalid

In [5]:
filename

'cnyes-2024-11-01~2024-11-16.csv'

In [6]:
pd.read_csv('Forex_dashboard/原始鉅亨/'+ filename)

,2024-11-15,18:05,"美匯指數報106.548,關注美國10月工業及零售數據",0,外匯,Unnamed: 5,Unnamed: 6,美匯指數 ==== 最新美元匯價 ====== *上述報價只供參考用,https://news.cnyes.com/news/id/5781268
0,2024-11-15,17:18,台灣續列匯率操縱觀察名單 央行回應美方2項建議,1,台股,台股新聞,"匯率,央行,美元,台幣,貿易順差,經常帳",美國財政部公布半年度匯率政策報告 (檢視期間為 2023 年下半年至 2024 年上半年)，...,https://news.cnyes.com/news/id/5781113
1,2024-11-15,17:06,"《神州經脈》消費增速勝預期,樓價環比跌幅縮,滬指周挫逾3%",0,A股港股,大陸政經,NaN,"《神州經脈》國家統計局今日(15日)發布的一系列經濟數據有好有壞,滬綜指震盪調整,全日收跌1...",https://news.cnyes.com/news/id/5781159
2,2024-11-15,17:05,"英鎊兌美元1.2663,英第三季GDP按季增長0.1%遜預期",0,外匯,NaN,NaN,美匯指數 ==== 最新美元匯價 ...,https://news.cnyes.com/news/id/5781147
3,2024-11-15,16:44,"【聚焦人幣】人幣兌一美元即期收升99點子,報7.2310",0,A股港股,大陸政經,NaN,"《經濟通通訊社15日專訊》兌一美元即期匯率(CNY)今日收報7.2310,較上個交易日4時3...",https://news.cnyes.com/news/id/5781089
4,2024-11-15,16:40,"《財資快訊》美電升報7.7833,美元兌人幣軟報7.2313",0,A股港股,港股,NaN,《財資快訊》美電軟報7.7818╱7847。美元兌升報7.2311╱2314。一年期不交收益...,https://news.cnyes.com/news/id/5781083
...,...,...,...,...,...,...,...,...,...
215,2024-11-01,09:37,"【聚焦人幣】人幣兌一美元即期開市升21點子,報7.1139",0,A股港股,大陸政經,NaN,"《經濟通通訊社1日專訊》兌一美元中間價今報7.1135,為10月25日以來最高,較上個交易日...",https://news.cnyes.com/news/id/5760155
216,2024-11-01,09:21,"【聚焦人幣】人幣中間價升115點子,報7.1135",0,外匯,NaN,NaN,"《經濟通通訊社1日專訊》中國外匯交易中心兌一美元中間價今報7.1135,為10月25日以來最...",https://news.cnyes.com/news/id/5760143
217,2024-11-01,09:03,"歐元兌美元報1.0887,歐元區10月CPI按年升2.0%",0,外匯,NaN,NaN,美匯指數 ==== 最新美元匯價 ...,https://news.cnyes.com/news/id/5760071
218,2024-11-01,08:59,【遠東商銀】關注美國就業報告，美元漲跌互見,0,外匯,NaN,NaN,◆USD/JPY美國第三季國內生產總值 (GDP) 環比增長年率為 2.8%、消費者支出以一...,https://news.cnyes.com/news/id/5749315


In [2]:
url = 'https://www.macromicro.me/charts/117199/global-share-of-international-payments-via-swift-by-major-currency'
driver = uc.Chrome()
driver.get(url)
time.sleep(5)
data = driver.find_elements(By.XPATH,'//*[@id="ccApp"]/div/div[2]/div[1]/div/div/div[2]/div[4]/ul')[0].text.split()
df = {
    '幣別':[],
    '最新公告時間':[],
    '占比':[],
    # '前一期':[]
}
for i in range(0,len(data), 4):
    df['幣別'].append(data[i])
    df['最新公告時間'].append(data[i+1])
    df['占比'].append(data[i+2])
    # df['前一期'].append(data[i+3])

df = pd.DataFrame(df)
test = [float(tmp[:-1]) for tmp in df.iloc[1:]['占比']]
df.iloc[0]['占比'] = str(round(100-np.sum(test),2))+'%'
df.iloc[0]['最新公告時間'] = df.iloc[1]['最新公告時間']

In [139]:
jsonFile = open("auto/data/12_SWIFT各幣別支付占比.json",'r')
f =  jsonFile.read()   # 要先使用 read 讀取檔案
df12 = json.loads(f)      # 再使用 loads
jsonFile.close()
df12_labels = list(tmp for tmp in df12.keys())

In [144]:
df12['人民幣']['時間'][-1]

'2024-09'

In [146]:
df12['人民幣']['SWIFT占比'][-1]

'3.61%'

In [133]:
updatepath = 'auto/data/'
path_update = f'{updatepath}9_1原料價格.json'
jsonFile = open(path_update,'r')
f =  jsonFile.read()   # 要先使用 read 讀取檔案
df_update = json.loads(f)
jsonFile.close()

In [137]:
df_update['Non-ferrous']['LME-Aluminium'].keys()

dict_keys(['LME Aluminium Official Prices', 'LME Aluminium Closing Prices', 'LME Aluminium opening stocks', 'LME Aluminium Asian Reference Price'])

In [138]:
df_update['Non-ferrous']['LME-Aluminium']['LME Aluminium Official Prices']

{'date': ['2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/12',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/13',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14',
  '2024/11/14'],
 'CONTRACT': ['Cash',
  '3-month',
  'Dec-25',
  'Dec-26',
  'Dec-27',
  'Cash',
  '3-month',
  'Dec-25',
  'Dec-26',
  'Dec-27',
  'Cash',
  '3-month',
  'Dec-25',
  'Dec-26',
  'Dec-27'],
 'BID': ['2528.00',
  '2564.00',
  '2608.00',
  '2643.00',
  '2657.00',
  '2513.50',
  '2549.00',
  '2593.00',
  '2628.00',
  '2653.00',
  '2503.00',
  '2524.00',
  '2572.00',
  '2617.00',
  '2652.00'],
 'OFFER': ['2530.00',
  '2565.00',
  '2613.00',
  '2648.00',
  '2662.00',
  '2514.00',
  '2549.50',
  '2598.00',
  '2633.00',
  '2658.00',
  '2504.00',
  '2526.00',
  '2577.00',
  '2622.00',
  '2657.00']}

In [127]:
querys = ['Currency', 'Currency_date','Currency_unit', 'Stock Market', 'Stock Market_date', 'Stock Market_unit']
for key in list(data.keys()):
    for query in querys:
        data[key][f'{query}'] = data[key][f'{query}'][-2:]

In [129]:
json_data = json.dumps(data)
with open(path_update, "w") as json_file:
    json_file.write(json_data)

In [116]:
now = arrow.now()

# 轉換成指定格式 Sep/24 時:分:秒
formatted_time = now.format("DD/MMM/YY HH:mm:ss")

print(formatted_time)

15/Nov/24 14:42:51


In [113]:
# updated = request.form.get('updated_item')  # Retrieve updated_item from the form
updated = 'all'

GEOs = ['china', 'india', 'malaysia', 'turkey', 'united-states']
filepath = 'auto/new_data/'

for GEO in GEOs:
    options = Options() 
    # options.add_argument('--headless=new')  # 啟動Headless 無頭
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options = options)
    driver.get(f'https://tradingeconomics.com/{GEO}/indicators')
    updated_items = driver.find_elements(By.XPATH, '//*[@id="pagemenutabs"]')[0].text.split()
    time.sleep(3)
    macro = driver.find_elements(By.XPATH, '//*[@id="pagemenutabs"]')[0].text.split('\n')[:-1]
    rows = []
    for l in range(0, len(macro)):
        driver.find_element(By.XPATH, f'//*[@id="pagemenutabs"]/li[{l+1}]/a').click()
        data = driver.find_elements(By.XPATH, f'//*[@id="{macro[l].lower()}"]/div/div/table/tbody')[0].text.replace('30 Year Mortgage Rate', 'thirty Year Mortgage Rate').replace('15 Year Mortgage Rate', 'fifteen Year Mortgage Rate').replace('Michigan 5 Year Inflation Expectations', 'Michigan five Year Inflation Expectations').replace('4-week','four-week').replace('1 Year MLF Rate', 'one Year MLF Rate').replace('14-Day Reverse Repo Rate', 'fourteen-Day Reverse Repo Rate').replace('Loan Prime Rate 5Y', 'Loan Prime Rate five-Y').split('\n')

        for i in range(0,len(data)):
            tmp = data[i].split(' ')
            name = ''
            unit = ''
            notice_date = ''
            last = ''
            previous = ''
            highest = ''
            lowest = ''
            signal = 0
            pre = 0
            for j in range(0,len(tmp)):
                # j = pre
                if signal == 0:
                    if ord(tmp[j][0])>=65: # name
                        name += tmp[j]+' '
                    else: # 數字們
                        signal = 1
                        k = j

                if signal == 1 and ord(tmp[j][0])>=65:
                    if '/' not in tmp[j]:
                        unit += tmp[j]+' '
                    else:
                        signal = 2
                        pre = j
                if signal == 2:
                    if '/' in tmp[j]:
                        notice_date = tmp[j]
                
                
            name = name[:-1]
            unit = unit[:-1]

            try:
                if '/' not in tmp[k+1] and ord(tmp[k+1][0])<65:
                    previous += tmp[k+1]
                else:
                    previous = ''
            except:
                previous = ''
            
            try:
                if '/' not in tmp[k+2] and ord(tmp[k+2][0])<65:
                    highest += tmp[k+2]
            except:
                highest = ''
            
            try:
                if '/' not in tmp[k+3]and ord(tmp[k+3][0])<65:
                    lowest += tmp[k+3]
            except:
                lowest = ''
            
            if 'Stock' in name or 'Currency' in name:
                # 取得當前時間
                now = arrow.now()

                # 轉換成指定格式 Sep/24 時:分:秒
                formatted_time = now.format("MMM/DD HH:mm:ss")

                rows.append({'總經種類':macro[l],'指標名稱':name, 'Last':tmp[k], 'Previous': previous, 'Higest': highest, 'Lowest': lowest, '單位': unit, '公告日期': formatted_time})
            else:    
                rows.append({'總經種類':macro[l],'指標名稱':name, 'Last':tmp[k], 'Previous': previous, 'Higest': highest, 'Lowest': lowest, '單位': unit, '公告日期': notice_date})
    data = pd.DataFrame(rows)

In [114]:
data

,總經種類,指標名稱,Last,Previous,Higest,Lowest,單位,公告日期
0,Overview,Currency,107,107,,,,Nov/15 14:38:07
1,Overview,Stock Market,5927,5949,,,points,Nov/15 14:38:07
2,Overview,GDP Growth Rate,2.8,3,,,percent,Sep/24
3,Overview,GDP Annual Growth Rate,2.7,3,,,percent,Sep/24
4,Overview,Unemployment Rate,4.1,4.1,,,percent,Oct/24
...,...,...,...,...,...,...,...,...
347,Energy,Refinery Crude Runs,175,281,,,Thousand Barrels,Nov/24
348,Health,Hospital Beds,2.77,2.78,,1000,per people,Dec/21
349,Health,Hospitals,18.55,19.11,,,per one million people,Dec/19
350,Health,Medical Doctors,2.77,2.74,,1000,per people,Dec/19


In [3]:
df = get_data4()
df.to_excel('auto/new_data/4_政府推動計畫名單.xlsx', index = 0)

100%|██████████| 89/89 [01:03<00:00,  1.41it/s]


In [ ]:
# df = pd.read_excel('auto/new_data/4_政府推動計畫名單.xlsx')

# try:
#     updatepath = 'auto/data/'
#     path_update = f'{updatepath}4_政府推動計畫名單.xlsx'
#     DF = pd.read_excel(path_update)
# except:
#     DF = pd.DataFrame(columns=['公司名稱', '計畫名稱', '核定日期'])

# for i in range(0,len(df)):
#     if df.iloc[i].values not in DF.values:
#         DF = pd.concat([DF, pd.DataFrame(df.iloc[i]).T], axis = 0)

# DF = DF.sort_values('核定日期', ignore_index=True)
# DF.to_excel(path_update, index = 0)

0
9
16
28
55
65
72
79
88
99
109
115
125
139
145
152
155
160
164
170
172
182
188
198
206
218
228
238
246
254
258
268
278
290
301
311
322
333
345
349
351
359
366
375
386
394
404
419
429
441
451
458
468
474
480
491
498
501
511
523
525
529
531
533
540
550
558
566
578
603
613
623
626
628
632
639
640
650
657
664
667
671
679
690
697
708
716
721
728
734
740
748
756
762
765
767
773
777
782
787
795
803
809
819
826
834
839
848
853
860
865
873
884


In [105]:
# 讀取新的資料
df = pd.read_excel('auto/new_data/4_政府推動計畫名單.xlsx')

# 更新檔案路徑
updatepath = 'auto/data/'
path_update = f'{updatepath}4_政府推動計畫名單.xlsx'

# 嘗試讀取舊資料，若檔案不存在，則建立空的 DataFrame
try:
    DF = pd.read_excel(path_update)
except FileNotFoundError:
    DF = pd.DataFrame(columns=['公司名稱', '計畫名稱', '核定日期'])

# 合併新資料與舊資料，並去除重複資料
DF = pd.concat([DF, df], ignore_index=True).drop_duplicates(subset=['公司名稱', '計畫名稱', '核定日期'])

# 依照 '核定日期' 排序
DF = DF.sort_values(by='核定日期', ignore_index=True)

# 儲存更新後的資料
DF.to_excel(path_update, index=False)


In [106]:
DF

,公司名稱,計畫名稱,核定日期
0,富強鑫機密工業股份有限公司,iSCM-富強鑫AI創鑫應用計畫,2019/01/30
1,芳德鑄鋁股份有限公司,鋁製家具自動化產線與智慧供應鏈建置計畫,2019/01/30
2,聚和國際股份有限公司,精密化學應用生技產業-智造聚和雲供應鏈平台,2019/01/30
3,佳凌科技股份有限公司,精密光學產業高階車載鏡頭智慧供應鏈AI加值應用計畫,2019/01/30
4,路易莎職人咖啡股份有限公司,咖啡職人智慧化供應鏈應用計畫,2019/01/30
...,...,...,...
885,艾飛思科技股份有限公司,Top PCIe V6 64Gbps PAM4測試驗證平台開發計畫,2024/11/01
886,恆昌精密科技股份有限公司,NMP藥液回收蒸餾系統開發計畫,2024/11/01
887,陽程科技股份有限公司,先進半導體封裝狹縫式塗佈製程設備開發與驗證計畫,2024/11/01
888,均華精密工業股份有限公司,面板級封裝固晶製程設備,2024/11/01


In [92]:
df.drop_duplicates()

,公司名稱,計畫名稱,核定日期
0,筑波科技股份有限公司,化合物半導體晶圓品質檢驗技術及機台開發,2024/11/01
1,旺矽科技股份有限公司,矽光子晶圓測試系統開發,2024/11/01
2,天虹科技股份有限公司,矽光子異質封裝之12吋Auto Bonder整機驗證計畫,2024/11/01
3,鴻海精密工業股份有限公司,智載天下：生成式AI賦能智慧城市研訓計畫,2024/11/01
4,艾飛思科技股份有限公司,Top PCIe V6 64Gbps PAM4測試驗證平台開發計畫,2024/11/01
...,...,...,...
885,佳凌科技股份有限公司,精密光學產業高階車載鏡頭智慧供應鏈AI加值應用計畫,2019/01/30
886,聚和國際股份有限公司,精密化學應用生技產業-智造聚和雲供應鏈平台,2019/01/30
887,芳德鑄鋁股份有限公司,鋁製家具自動化產線與智慧供應鏈建置計畫,2019/01/30
888,路易莎職人咖啡股份有限公司,咖啡職人智慧化供應鏈應用計畫,2019/01/30


In [90]:
DF

,公司名稱,計畫名稱,核定日期
0,富強鑫機密工業股份有限公司,iSCM-富強鑫AI創鑫應用計畫,2019/01/30
1,大詠城機械股份有限公司,五軸加工具機底座之治金智慧化生產線系統建置研發計畫,2019/02/19
2,美利達工業股份有限公司,電動輔助全避震登山自行車設計與製程整合之創新技術開發計畫,2019/03/27
3,泓格科技股份有限公司,非侵入式智慧感知系統暨工業網路安全防護技術開發計畫,2019/04/26
4,幸福日子網路股份有限公司,WishDay體驗商孵化募資平台建置計畫,2019/05/13
...,...,...,...
108,群翊工業股份有限公司,智慧化懸吊式自動熱風爐驗證計畫,2024/09/26
109,穩脈科技股份有限公司,MHz/KHz雙輸出之超小型化MEMS振盪器ASIC,2024/10/11
110,永餘智能股份有限公司,智慧充儲系統應用於虛擬電廠平台,2024/10/15
111,友達光電股份有限公司、達擎股份有限公司及達運精密工業股份有限公司,友達智慧亞灣策略暨解決方案研發中心,2024/10/24


In [76]:
pd.DataFrame(df.iloc[i]).T

,公司名稱,計畫名稱,核定日期
1,旺矽科技股份有限公司,矽光子晶圓測試系統開發,2024/11/01


In [61]:
DF

[array([['穩脈科技股份有限公司', 'MHz/KHz雙輸出之超小型化MEMS振盪器ASIC', '2024/10/11'],
        ['矽眾科技股份有限公司', '先進極小尺寸高穩定度車用溫度補償石英振盪器ASIC晶片', '2024/10/11'],
        ['凌陽科技股份有限公司', '多模態大型語言模型加速器晶片滿足生成式AI邊緣應用研發計畫', '2024/10/11'],
        ...,
        ['芯鼎科技股份有限公司', '整合運算擴充晶粒對接技術之車載影像處理駕駛輔助域控制器系統晶片平台研發計畫',
         '2024/10/11'],
        ['耐能智慧股份有限公司', 'Kneron離線生成式AI Edge SoC', '2024/10/11'],
        ['訊灝晶片科技股份有限公司', '後量子加密高效能運算晶片安全計畫', '2024/10/11']], dtype=object),
 array(['筑波科技股份有限公司', '化合物半導體晶圓品質檢驗技術及機台開發', '2024/11/01'], dtype=object)]

In [10]:
DF

,公司名稱,計畫名稱,核定日期,0
0,穩脈科技股份有限公司,MHz/KHz雙輸出之超小型化MEMS振盪器ASIC,2024/10/11,NaN
1,矽眾科技股份有限公司,先進極小尺寸高穩定度車用溫度補償石英振盪器ASIC晶片,2024/10/11,NaN
2,凌陽科技股份有限公司,多模態大型語言模型加速器晶片滿足生成式AI邊緣應用研發計畫,2024/10/11,NaN
3,信驊科技股份有限公司,沉浸式影像AI視覺晶片,2024/10/11,NaN
4,瓦雷科技有限公司,實現大規模AI運算之NVMe-oC記憶體晶片設計,2024/10/11,NaN
...,...,...,...,...
879,耐能智慧股份有限公司,Kneron離線生成式AI Edge SoC,2024/10/11,NaN
880,訊灝晶片科技股份有限公司,後量子加密高效能運算晶片安全計畫,2024/10/11,NaN
公司名稱,NaN,NaN,NaN,筑波科技股份有限公司
計畫名稱,NaN,NaN,NaN,化合物半導體晶圓品質檢驗技術及機台開發
